# Hyperopt

http://hyperopt.github.io/hyperopt/

https://github.com/hyperopt/hyperopt
    
hyperopt is a Python library for optimizing over awkward search spaces with real-valued, discrete, and conditional dimensions.

## Algorithms
Currently two algorithms are implemented in hyperopt:

* Random Search
* Tree of Parzen Estimators (TPE)

Hyperopt has been designed to accommodate Bayesian optimization algorithms based on Gaussian processes and regression trees, but these are not currently implemented.

All algorithms can be run either serially, or in parallel by communicating via MongoDB.

## Installation


In [3]:
# pip install hyperopt -U
!pip freeze | grep hyperopt

hyperopt==0.1.2


# Basic tutorial

## FMin
https://github.com/hyperopt/hyperopt/wiki/FMin

This page is a tutorial on basic usage of hyperopt.fmin(). It covers how to write an objective function that fmin can optimize, and how to describe a search space that fmin can search.

Hyperopt的工作是在该函数的一组可能参数上找到标量值，可能是随机函数的最佳值。 虽然许多优化包将假设这些输入是从向量空间中提取的，但Hyperopt的不同之处在于它鼓励您更详细地描述搜索空间。 通过提供有关函数定义位置的更多信息以及您认为最佳值的位置，您可以允许hyperopt中的算法更有效地进行搜索。

The way to use hyperopt is to describe:

the objective function to minimize
the space over which to search
the database in which to store all the point evaluations of the search
the search algorithm to use

This (most basic) tutorial will walk through how to write functions and search spaces, using the default Trials database, and the dummy random search algorithm. Section (1) is about the different calling conventions for communication between an objective function and hyperopt. Section (2) is about describing search spaces.

Parallel search is possible when replacing the Trials database with a MongoTrials one; there is another wiki page on the subject of using mongodb for [parallel search](https://github.com/hyperopt/hyperopt/wiki/Parallelizing-Evaluations-During-Search-via-MongoDB).

Choosing the search algorithm is as simple as passing algo=hyperopt.tpe.suggest instead of algo=hyperopt.random.suggest. The search algorithms are actually callable objects, whose constructors accept configuration arguments, but that's about all there is to say about the mechanics of choosing a search algorithm.

### Defining a Function to Minimize

接下来的几节将介绍实现目标函数的各种方法，该目标函数最小化单个变量上的二次目标函数。在每个部分中，我们将搜索范围从-10到+10，我们可以用搜索空间来描述:

In [ ]:
space = hp.uniform('x', -10, 10)

#### The Simplest Case
The simplest protocol for communication between hyperopt's optimization algorithms and your objective function, is that your objective function receives a valid point from the search space, and returns the floating-point loss (aka negative utility) associated with that point.

In [4]:
from hyperopt import fmin, tpe, hp
best = fmin(fn=lambda x: x ** 2,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100)
print(best)

100%|██████████| 100/100 [00:00<00:00, 359.13it/s, best loss: 0.0002908558187163727]
{'x': -0.017054495557370576}


这个协议的优点是非常易读和快速输入。正如您所看到的，它几乎是一行代码。该协议的缺点是(1)这类函数不能将每次评价的额外信息返回到trials数据库，(2)这类函数不能与搜索算法或其他并发函数评价交互。在下面的示例中，您将看到为什么要做这些事情。

#### Attaching Extra Information via the Trials Object

如果您的目标函数很复杂并且需要很长时间才能运行，那么您几乎肯定希望保存更多的统计信息和诊断信息，而不仅仅是最后出现的一个浮点丢失。 对于这种情况，写入fmin函数来处理字典返回值。 我们的想法是，您的损失函数可以返回嵌套字典，其中包含您想要的所有统计信息和诊断信息。 实际情况不如说那么灵活：例如，当使用mongodb时，字典必须是有效的JSON文档。 仍然存在很多灵活性来存储特定于域的辅助结果。

当目标函数返回字典时，fmin函数在返回值中查找一些特殊的键值对，并将其传递给优化算法。 有两个必需的键值对：

status - 来自hyperopt.STATUS_STRINGS的键之一，例如'ok'表示成功完成，'fail'表示函数结果未定义。

loss - 您尝试最小化的浮点值函数值，如果状态为“ok”，则必须存在。

In [5]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(x):
    return {'loss': x ** 2, 'status': STATUS_OK }

best = fmin(objective,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100)

print (best)

100%|██████████| 100/100 [00:00<00:00, 393.31it/s, best loss: 6.648383363572952e-07]
{'x': -0.0008153761931509254}


#### The Trials Object

要真正看到返回字典的目的，让我们修改目标函数以返回更多内容，并将显式试验参数传递给fmin。

In [6]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def objective(x):
    return {
        'loss': x ** 2,
        'status': STATUS_OK,
        # -- store other results like this
        'eval_time': time.time(),
        'other_stuff': {'type': None, 'value': [0, 1, 2]},
        # -- attachments are handled differently
        'attachments':
            {'time_module': pickle.dumps(time.time)}
        }
trials = Trials()
best = fmin(objective,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100,
    trials=trials)

print (best)

100%|██████████| 100/100 [00:00<00:00, 398.26it/s, best loss: 0.001953503634490734]
{'x': -0.044198457376821806}


在本例中，对fmin的调用像以前一样进行，但是通过直接传递一个trials对象，我们可以检查在实验期间计算的所有返回值。

So for example:

* trials.trials - a list of dictionaries representing everything about the search
* trials.results - a list of dictionaries returned by 'objective' during the search
* trials.losses() - a list of losses (float for each 'ok' trial)
* trials.statuses() - a list of status strings


In [8]:
trials.results[:2]

[{'loss': 19.593353584080813,
  'status': 'ok',
  'eval_time': 1566366945.031306,
  'other_stuff': {'type': None, 'value': [0, 1, 2]}},
 {'loss': 82.77343216727205,
  'status': 'ok',
  'eval_time': 1566366945.032654,
  'other_stuff': {'type': None, 'value': [0, 1, 2]}}]

#### The Ctrl Object for Realtime Communication with MongoDB

### Defining a Search Space

搜索空间由嵌套函数表达式(包括随机表达式)组成。随机表达式是超参数。这个嵌套随机程序的抽样定义了随机搜索算法。超参数优化算法的工作原理是用自适应搜索策略代替常规的“采样”逻辑，不尝试实际采样搜索空间中指定的分布。

最好将搜索空间看作随机参数抽样程序。例如

In [9]:
from hyperopt import hp
space = hp.choice('a',
    [
        ('case 1', 1 + hp.lognormal('c1', 0, 1)),
        ('case 2', hp.uniform('c2', -10, 10))
    ])

运行此代码片段的结果是一个变量空间，它引用表达式标识符及其参数的图形。 什么都没有实际采样，它只是一个描述如何采样点的图表。 处理这种表达式图的代码在hyperopt.pyll中，我将这些图称为pyll图或pyll程序。

如果您愿意，可以通过从中抽样来评估样本空间。

In [13]:
import hyperopt.pyll.stochastic
print (hyperopt.pyll.stochastic.sample(space))
print (hyperopt.pyll.stochastic.sample(space))
print (hyperopt.pyll.stochastic.sample(space))
print (hyperopt.pyll.stochastic.sample(space))

('case 2', 2.26084123341008)
('case 2', 9.755311624097203)
('case 2', 6.900377220073739)
('case 1', 1.3121396556035247)


This search space described by space has 3 parameters:

* 'a' - selects the case
* 'c1' - a positive-valued parameter that is used in 'case 1'
* 'c2' - a bounded real-valued parameter that is used in 'case 2'

这里需要注意的一点是，每个可优化的随机表达式都有一个标签作为第一个参数。 这些标签用于将参数选择返回给调用者，并在内部以各种方式返回。

需要注意的第二件事是我们在图的中间使用了元组（在“case 1”和“case 2”的每一个周围）。 列表，词典和元组都升级为“确定性函数表达式”，以便它们可以成为搜索空间随机程序的一部分。

要注意的第三件事是数字表达式1 + hp.lognormal（'c1'，0,1），它嵌入到搜索空间的描述中。 就优化算法而言，在搜索空间中直接添加1和在目标函数本身的逻辑中添加1之间没有区别。 作为设计人员，您可以选择在何处进行此类处理，以实现您想要的模块化。 请注意，搜索空间内的中间表达式结果可以是任意Python对象，即使使用mongodb并行优化也是如此。 在搜索空间描述中添加新类型的非随机表达式很容易，请参阅下文（第2.3节）了解如何执行此操作。

需要注意的第四点是'c1'和'c2'是我们称之为条件参数的例子。 'c1'和'c2'中的每一个仅在返回的样本中表示特定值'a'。 如果'a'为0，则使用'c1'而不是'c2'。 如果'a'为1，则使用'c2'但不使用'c1'。 只要有意义，您应该以这种方式将参数编码为条件参数，而不是简单地忽略目标函数中的参数。 如果你揭露'c1'有时对目标函数没有影响的事实（因为它对目标函数的参数没有影响），那么搜索可以更有效地进行信用分配。

#### Parameter Expressions
目前hyperopt优化算法识别的随机表达式为

hp.choice(label, options)
hp.randint(label, upper)
Returns a random integer in the range [0, upper). 
* hp.uniform(label, low, high)
Returns a value uniformly between low and high.
* hp.quniform(label, low, high, q)
* hp.loguniform(label, low, high)
* hp.qloguniform(label, low, high, q)
* hp.normal(label, mu, sigma)
Returns a real value that's normally-distributed with mean mu and standard deviation sigma. When optimizing, this is an unconstrained variable.
* hp.qnormal(label, mu, sigma, q)
* hp.lognormal(label, mu, sigma)
* hp.qlognormal(label, mu, sigma, q)

#### A Search Space Example: scikit-learn

In [ ]:
from hyperopt import hp
space = hp.choice('classifier_type', [
    {
        'type': 'naive_bayes',
    },
    {
        'type': 'svm',
        'C': hp.lognormal('svm_C', 0, 1),
        'kernel': hp.choice('svm_kernel', [
            {'ktype': 'linear'},
            {'ktype': 'RBF', 'width': hp.lognormal('svm_rbf_width', 0, 1)},
            ]),
    },
    {
        'type': 'dtree',
        'criterion': hp.choice('dtree_criterion', ['gini', 'entropy']),
        'max_depth': hp.choice('dtree_max_depth',
            [None, hp.qlognormal('dtree_max_depth_int', 3, 1, 1)]),
        'min_samples_split': hp.qlognormal('dtree_min_samples_split', 2, 1, 1),
    },
    ])

# hyperopt和gridsearch的简单比较

之前做参数选择用的是sklearn的gridsearch，对比hyperopt，发现后者有其独特的优势在于它更适合做所有模型的参数检测的，因为gridsearch主要是配合sklearn的model来设计的，如果model不是sklearn的则不能自动支持。而hyperopt可以适应各种模型，只是自己要写损失函数和cv方法。

Hyperopt在自己写好配套代码后，运行发现其输出的参数配对，比gridsearch更倾向于找优化效果更好的参数配对，配对的取值不用事先给定。而gridsearch则是用所有给定参数取值的全排列，需要事先把各个参数的取值列表给定。Hyperopt相对于后者，更能挖掘更好的参数配对。

下面的例子是对xgboost的eta和maxdepth两个参数做的参数检测例子。



In [ ]:
from hyperopt import fmin,tpe,hp,partial


###
### hyperopt do something like gridsearch.
###
space = {
         "eta":hp.uniform("eta",0.01, 0.1),
         "max_depth":hp.choice("max_depth",range(1,10)),
        }
algo = partial(tpe.suggest,n_startup_jobs=1)
best = fmin(xgboost_hyperopt,space,algo = algo,max_evals=50)
print("best param:", best)
score = xgboost_hyperopt(best)
print("xgboost_hyperopt score:",score)


df = pd.read_csv("../input/hypteropt_trace.csv")
df.head()
df.plot.scatter(x='eta', y='maxdepth');



从eta和maxdepth的取值范围散点图来看，覆盖了大部分交叉范围，但是又不是简单的平均分配的配对样本。对于性能提升的方向，试探的步长更细，样本更多。这是与gridsearch不同的地方。
